# Limitation of sklearn’s non-negative matrix factorization library

Upload a Jupyter notebook with a mix of code/markdown to answer the following questions. Please mark the sections of your notebook as 1 and 2 so that graders can follow along.


#### Table of Contents

- [Part 1](#Part-1)
- [Part 2](#Part-2)


## Part 1

1. Load the movie ratings data (as in the HW3-recommender-system) and use matrix factorization technique(s) and predict the missing ratings from the test data. Measure the RMSE. You should use sklearn library. [10 pts]

Make sure that your notebook includes the following: use's sklearn's non-negative matrix factorization ,notebook shows the RMSE with an analysis of what that RMSE means


### 1. Use Matrix Factorization Techniques and Predict the Missing Ratings from the Test Data

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load the movie ratings data from module 3
users = pd.read_csv('data/users.csv')
movies = pd.read_csv('data/movies.csv')
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

print("Train columns:", train.columns.tolist())
print("Test columns:", test.columns.tolist())


Train columns: ['uID', 'mID', 'rating']
Test columns: ['uID', 'mID', 'rating']


In [ ]:
print("Dataset Overview:")
print(f"Training samples: {len(train)}")
print(f"Test samples: {len(test)}")
print(f"Number of users: {len(users)}")
print(f"Number of movies: {len(movies)}")
print(f"Rating range: {train['rating'].min()} - {train['rating'].max()}")


Dataset Overview:


NameError: name 'train' is not defined

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [ ]:
# Check for data sparsity
total_possible_ratings = len(users) * len(movies)
actual_ratings = len(train)
sparsity = (1 - actual_ratings / total_possible_ratings) * 100
print(f"Data sparsity: {sparsity:.2f}%")

Data sparsity: 97.01%


Here we have a high sparsity, which is normal, but the method filling missing values will likely change RMSE a lot.

### Creating the User Item Rating Matrix

In [ ]:
# Create user-item matrix from training data
def create_user_item_matrix(ratings_df, users_df, movies_df):
    """
    Create a user-item rating matrix from the ratings dataframe
    """
    # Create mappings for user and movie IDs to matrix indices
    user_to_idx = {user_id: idx for idx, user_id in enumerate(users_df['uID'])}
    movie_to_idx = {movie_id: idx for idx, movie_id in enumerate(movies_df['mID'])}
    
    # Create the matrix
    n_users = len(users_df)
    n_movies = len(movies_df)
    
    user_item_matrix = np.zeros((n_users, n_movies))
    
    for _, row in ratings_df.iterrows():
        user_idx = user_to_idx[row['uID']]
        movie_idx = movie_to_idx[row['mID']]
        user_item_matrix[user_idx, movie_idx] = row['rating']
    
    return user_item_matrix, user_to_idx, movie_to_idx

# Create the training matrix
train_matrix, user_to_idx, movie_to_idx = create_user_item_matrix(train, users, movies)
print(f"User-item matrix shape: {train_matrix.shape}")
print(f"Non-zero entries: {np.count_nonzero(train_matrix)}")
print(train_matrix.size)

User-item matrix shape: (6040, 3883)
Non-zero entries: 700146
23453320


Once again high sparsity is comfirmed.

### Applying NMF

In [ ]:
# Apply NMF to the user-item matrix
def apply_nmf(matrix, n_components=50, random_state=42):
    """
    Apply Non-Negative Matrix Factorization to the user-item matrix
    """
    # Convert to sparse matrix for efficiency
    sparse_matrix = csr_matrix(matrix)
    
    # Initialize and fit NMF model
    nmf_model = NMF(
        n_components=n_components,
        random_state=random_state,
        init='random',
        max_iter=500,
        alpha_W=0.1,
        alpha_H=0.1,
        l1_ratio=0.0
    )
    
    # Fit the model (only on non-zero entries)
    W = nmf_model.fit_transform(matrix)
    H = nmf_model.components_
    
    # Reconstruct the full matrix
    reconstructed_matrix = np.dot(W, H)
    
    return nmf_model, W, H, reconstructed_matrix

# Apply NMF with different numbers of components
n_components_list = [10, 25, 50, 100]
results = {}

for n_comp in n_components_list:
    print(f"\nTesting NMF with {n_comp} components...")
    nmf_model, W, H, reconstructed = apply_nmf(train_matrix, n_components=n_comp)
    
    # Calculate reconstruction error on training data (only for non-zero entries)
    mask = train_matrix > 0
    train_rmse = sqrt(mean_squared_error(
        train_matrix[mask], 
        reconstructed[mask]
    ))
    
    results[n_comp] = {
        'model': nmf_model,
        'W': W,
        'H': H,
        'reconstructed': reconstructed,
        'train_rmse': train_rmse
    }
    
    print(f"Training RMSE: {train_rmse:.4f}")


Testing NMF with 10 components...
Training RMSE: 3.3515

Testing NMF with 25 components...
Training RMSE: 3.3515

Testing NMF with 50 components...
Training RMSE: 3.3515

Testing NMF with 100 components...
Training RMSE: 3.3515


### Making Prediction on Test Data

In [ ]:
def predict_test_ratings(test_df, reconstructed_matrix, user_to_idx, movie_to_idx):
    """
    Predict ratings for test data using the reconstructed matrix
    """
    predictions = []
    actual_ratings = []
    
    for _, row in test_df.iterrows():
        user_id = row['uID']
        movie_id = row['mID']
        actual_rating = row['rating']
        
        # Get indices
        if user_id in user_to_idx and movie_id in movie_to_idx:
            user_idx = user_to_idx[user_id]
            movie_idx = movie_to_idx[movie_id]
            
            predicted_rating = reconstructed_matrix[user_idx, movie_idx]
            
            # Clip predictions to valid rating range
            predicted_rating = np.clip(predicted_rating, 1, 5)
            
            predictions.append(predicted_rating)
            actual_ratings.append(actual_rating)
        else:
            # Handle users/movies not in training data with global mean
            global_mean = train['rating'].mean()
            predictions.append(global_mean)
            actual_ratings.append(actual_rating)
    
    return np.array(predictions), np.array(actual_ratings)

# Evaluate all models on test data
print("Test Set Evaluation:")
print("=" * 50)

best_rmse = float('inf')
best_n_components = None

for n_comp in n_components_list:
    predictions, actual = predict_test_ratings(
        test, 
        results[n_comp]['reconstructed'], 
        user_to_idx, 
        movie_to_idx
    )
    
    test_rmse = sqrt(mean_squared_error(actual, predictions))
    results[n_comp]['test_rmse'] = test_rmse
    results[n_comp]['predictions'] = predictions
    results[n_comp]['actual'] = actual
    
    print(f"NMF with {n_comp} components:")
    print(f"  Test RMSE: {test_rmse:.4f}")
    print(f"  Training RMSE: {results[n_comp]['train_rmse']:.4f}")
    print()
    
    if test_rmse < best_rmse:
        best_rmse = test_rmse
        best_n_components = n_comp

print(f"Best performance: {best_n_components} components with RMSE = {best_rmse:.4f}")

Test Set Evaluation:
NMF with 10 components:
  Test RMSE: 2.7743
  Training RMSE: 3.3515

NMF with 25 components:
  Test RMSE: 2.7743
  Training RMSE: 3.3515

NMF with 50 components:
  Test RMSE: 2.7743
  Training RMSE: 3.3515

NMF with 100 components:
  Test RMSE: 2.7743
  Training RMSE: 3.3515

Best performance: 100 components with RMSE = 2.7743


### RMSE Analysis 

In [ ]:
best_model = results[best_n_components]

print("RMSE Analysis:")

print(f"Best Test RMSE: {best_rmse:.4f}")
print()

baseline_rmse = 1.264
print(f"NMF improvement over baseline: {((baseline_rmse - best_rmse) / baseline_rmse * 100):.1f}")

RMSE Analysis:
Best Test RMSE: 2.7743

NMF improvement over baseline: -119.5


Here we see that the best RMSE is 2.774, which is very poor, and represent a high prediction error. This is likely due to substiting 0 for null/empty values. Considering RMSE/4 * 100 should represent what the error is relative to the 1-5 range for our scoring system for movies; the error here is 69%. This represents a extreme decline in performance in NMF over a baseline of 1.264 (From module 3) of 119.5%.

## Part 2 
Analysis of Result and Limitations

Discuss the results and why sklearn's non-negative matrix facorization library did not work well compared to simple baseline or similarity-based methods we’ve done in Module 3. Can you suggest a way(s) to fix it? [10 pts]

The performance of sklearn's NMF is extremely poor in comparision to Module 3 Assignment performance. I achieved and RMSE score of 2.774 which is worse than every baseline, collaborative and jaccard method.

In [ ]:
def improved_nmf(matrix, n_components=50):
    """
    NMF with stronger regularization and better init
    """
    nmf_improved = NMF(
        n_components=20,     # Try more factors
        init='nndsvda',
        solver='mu',         # Try the other solver
        max_iter=2000,
        alpha_W=0.05,        # Slightly lighter regularization
        alpha_H=0.05,
        l1_ratio=0.1,
        beta_loss='frobenius',
        random_state=42
    )
    return nmf_improved

# === Train Improved NMF ===
nmf_model_improved = improved_nmf(train_matrix, n_components=best_n_components)  # reuse best n_components

W_improved = nmf_model_improved.fit_transform(train_matrix)
H_improved = nmf_model_improved.components_

reconstructed_improved = np.dot(W_improved, H_improved)

# === Predict test ratings ===
predictions_improved, actual_improved = predict_test_ratings(
    test,
    reconstructed_improved,
    user_to_idx,
    movie_to_idx
)

improved_rmse = sqrt(mean_squared_error(actual_improved, predictions_improved))


print(f"Global Mean Baseline RMSE: {baseline_rmse:.4f}")
print(f"Original Best NMF RMSE:    {best_rmse:.4f}")
print(f"Improved NMF RMSE:         {improved_rmse:.4f}\n")

Global Mean Baseline RMSE: 1.2640
Original Best NMF RMSE:    2.7743
Improved NMF RMSE:         2.7276



#### Improved NMSE!


Through some hypertuning I was able to slighly improve the performance of the NMF method. Albeit not by much. NMF did not perform well compared to baseline methods as sklearn.NMF does not support missing data, and treats 0s as real data points despite our scale being 1-5. From the EDA we can see the data is mostly empty. Global baseline mean makes no assumptions and just uses overall average. 

Can you suggest a way(s) to fix it?

The simplest answer is not the use sklearn.NMF, alternatively I could attmept something like jaccard method where I subsitute all missing values for the median in the range of values. I think the best solution here is to use a different library.

|Method|RMSE|
|:----|:--------:|
|Baseline, $Y_p$=3| 1.264|
|Baseline, $Y_p=\mu_u$| 1.035|
|Content based, item-item| 1.0128|
|Collaborative, cosine| 1.0263|
|Collaborative, jaccard, $M_r\geq 3$| 0.9819 |
|Collaborative, jaccard, $M_r\geq 1$|  0.9914|
|Collaborative, jaccard, $M_r$|  0.9509|
|NMF| 2.7743 |
|NMF improved| 2.7276 |

Here I have added to the chart from module 3 our new RMSE.